# Comparison Recipes

In [40]:
import json
from icecream import ic

In [41]:
with open("../data/items.json") as f:
    items = json.load(f)

with open("../data/recipes.json") as f:
    recipes = json.load(f)

with open("../data/resources.json") as f:
    resources = json.load(f)

In [42]:
def find_recipes_with_product(item: str) -> dict:
    return {
        slug: recipe
        for slug, recipe in recipes.items()
        if any(products["item"] == item for products in recipe["products"])
    }

In [43]:
def find_cycle(item: str, visisted_recipes=[]):
    selected_recipes = find_recipes_with_product(item)

    for slug, recipe in selected_recipes.items():
        ic(slug)
        if slug in visisted_recipes:
            ic("cycle found", visisted_recipes)
            ic(recipes[visisted_recipes[0]])
            return visisted_recipes

        visisted_recipes.append(slug)

        for ingredient in recipe["ingredients"]:
            item_name = ingredient["item"]
            ic(item_name)
            if item_name in list(resources.keys()):
                continue
            return find_cycle(item_name, visisted_recipes)
        break


find_cycle("water")

ic| slug: 'recipe-unpackagewater'
ic| item_name: 'packaged-water'
ic| slug: 'recipe-packagedwater'
ic| item_name: 'water'
ic| item_name: 'empty-canister'
ic| slug: 'recipe-unpackageturbofuel'
ic| item_name: 'packaged-turbofuel'
ic| slug: 'recipe-packagedturbofuel'
ic| item_name: 'turbofuel'
ic| slug: 'recipe-alternate-turbofuel'
ic| item_name: 'fuel'
ic| slug: 'recipe-liquidfuel'
ic| item_name: 'crude-oil'


In [67]:
def get_cost(item: str, visited=[]) -> float:
    selected_recipes = find_recipes_with_product(item)

    cost_of_recipes = []
    for slug, recipe in selected_recipes.items():
        if slug in visited:
            return None, -1

        visited.append(slug)

        recipe_cost = 0
        for ingredient in recipe["ingredients"]:
            item_name = ingredient["item"]

            if item_name in list(resources.keys()):
                cost = ingredient["amount_per_minute"] * resources[item_name]["rarity"]
                recipe_cost += cost
            else:
                _, child_cost = get_cost(item_name, visited)
                recipe_cost += child_cost

        output_per_minute = [p for p in recipe["products"] if p["item"] == item][0][
            "amount_per_minute"
        ]
        cost_of_recipes.append((slug, recipe_cost / output_per_minute))

    cost_of_recipes = [(slug, cost) for slug, cost in cost_of_recipes if cost >= 0]
    cost_of_recipes.sort(key=lambda r: r[1])

    if cost_of_recipes:
        return cost_of_recipes[0]
    return [], -1


get_cost("silica")

('recipe-alternate-silica-distilled', 0.2944726151998446)